# [Create Agent with Function Call](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-function-call) using Tools and Tool Resources
You can enhance your Agents by defining **callback functions as function tools**. These can be provided to `create_agent` via either the `toolset` parameter or the **combination** of `tools and tool_resources`. Here are the distinctions:

1. **toolset**: When using the toolset parameter, you provide not only the function definitions and descriptions but also their implementations. The SDK will execute these functions within `create_and_run_process` or `streaming`. These functions will be invoked based on their definitions.
2. **tools and tool_resources**: When using the tools and tool_resources parameters, only the function definitions and descriptions are provided to `create_agent`, without the implementations. The `Run` or event handler of stream will raise a `requires_action` status based on the function definitions. **Your code must handle this status and call the appropriate functions**.<br/>

As a reference point, let's see how we were used to managed this with [OpenAI Assistants API](https://github.com/maurominella/openai/blob/main/assistantapi/Assistant%20APIs%2002%20-%20SDK%20with%20Function%20Calling.ipynb)

In [1]:
import os, time
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet, ToolOutput # <<<<<<<<<<<<<<< SPECIFIC FOR FUNCTION CALLING
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
# define some custom functions

from datetime import date

def save_file(joke:str) -> str:
    """
    Saves a text file using the textual content passed in the joke variable.
    Args:
        joke (str): The text content to be saved into the file. 
        
    Returns:
        Optional[str]: Returns 'success: file saved' on success, otherwise returns an error message.
    """
    file_name = "joke.txt"
    try:
        with open(file_name, "w") as f:
            f.write(joke)
        return "success: file saved"
    except Exception as e:
        return f"error: {str(e)}"


def get_flights(date_1:date, date_2:date) -> str:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)


def my_cat_born_date() -> str:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)


def send_email(to:str, subject:str, body:str) -> str:
    """ Sends an email """
    import requests, json
    url = 'https://prod-18.swedencentral.logic.azure.com:443/workflows/4f7a19b041e04a9e8ea47303e1af503c/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=TX-eDahoU_QIEOjw9qOXjRyPNqA9s4IVkd0osbsyzzI'  

    headers = {
        'Content-Type': 'application/json'
    }

    data = {
        "to": to,
        "subject": subject,
        "body": body
    }
    response = {"response": str(requests.post(url, headers=headers, data=json.dumps(data)))}
    return json.dumps(response)

In [4]:
from typing import Any, Callable, Set

user_functions: Set[Callable[..., Any]] = {
    save_file, 
    get_flights, 
    my_cat_born_date,
    send_email
}

user_functions

{<function __main__.get_flights(date_1: datetime.date, date_2: datetime.date) -> str>,
 <function __main__.my_cat_born_date() -> str>,
 <function __main__.save_file(joke: str) -> str>,
 <function __main__.send_email(to: str, subject: str, body: str) -> str>}

In [5]:
# Let's use the "user_function" set, to call its functions

for function in user_functions:    
    if function.__name__ == "my_cat_born_date":
        result = function()
    elif function.__name__ == "save_file": 
        result = function("This is the content")
    elif function.__name__ == "get_flights": 
        result = function("2015-01-01", "2021-04-04")
    elif function.__name__ == "send_email": 
        result = function("mauromi@microsoft.com", "email from Python Agent", "body")

    print(f"{function.__name__}() returned {result}")

send_email() returned {"response": "<Response [200]>"}
get_flights() returned {"flights": 2285}
my_cat_born_date() returned {"cat_born_date": "2015-01-02"}
save_file() returned success: file saved


In [6]:
functions = FunctionTool(user_functions)
functions.definitions

[{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}}},
 {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string', 'description': 'No description'}}, 'required': ['date_1', 'date_2']}}},
 {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content 

In [7]:
# Check if we already have any agents defined
project_client.agents.list_agents()

{'object': 'list', 'data': [], 'first_id': None, 'last_id': None, 'has_more': False}

In [8]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=functions.definitions
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_1TkHoUprIX7Xctg60VJ7sIPB', 'object': 'assistant', 'created_at': 1735850045, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string', 'description': 'No description'}}, 'required': ['date_1', 'date_2']}, 'strict': False}}, {'type'

In [9]:
# Check again if the agent created above is now present here
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_1TkHoUprIX7Xctg60VJ7sIPB', 'object': 'assistant', 'created_at': 1735850045, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string', 'description': 'No description'}}, 'required': ['date_1', 'date_2']}, 'strict': False}}, {'type': 'function',

In [10]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

Created thread: {'id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'object': 'thread', 'created_at': 1735850046, 'metadata': {}, 'tool_resources': {}}


In [11]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also."
)
print(f"Created message: {message}")

Created message: {'id': 'msg_IYoAv3Hkn75bGvV5e8PLwqLV', 'object': 'thread.message', 'created_at': 1735850047, 'assistant_id': None, 'thread_id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [12]:
%%time

run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
print(f"Run: {run}")

Run: {'id': 'run_8qHbfxr3tSfETSMHZwgWfJBM', 'object': 'thread.run', 'created_at': 1735850048, 'assistant_id': 'asst_1TkHoUprIX7Xctg60VJ7sIPB', 'thread_id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'status': 'in_progress', 'started_at': 1735850048, 'expires_at': 1735850648, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'p

In [13]:
# Check run status
project_client.agents.get_run(thread_id=thread.id, run_id=run.id).status

<RunStatus.REQUIRES_ACTION: 'requires_action'>

# Please run this cells only if <RunStatus.REQUIRES_ACTION: 'requires_action'>

In [16]:
# just for checking: analyze the current status

run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
print(f"Current run status: {run.status}")
print(f"\nRequired action(s): {run.required_action}")
print(f"\nWe need to run {len(run.required_action.submit_tool_outputs.tool_calls)} tool call(s): {run.required_action.submit_tool_outputs.tool_calls}")

i = 0
tool_outputs = []
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    i += 1
    output = functions.execute(tool_call)
    print(f"\n{i} - Executing tool_call {tool_call.function.name} ({tool_call.id}) >>> output: {output}")
    tool_outputs.append(
        ToolOutput(
            tool_call_id=tool_call.id,
            output=output
        )
    )
    
run = project_client.agents.submit_tool_outputs_to_run(
    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
)

while run.status in ["queued", "in_progress"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
    print(f"Current run status: {run.status}")

Current run status: RunStatus.REQUIRES_ACTION

Required action(s): {'type': 'submit_tool_outputs', 'submit_tool_outputs': {'tool_calls': [{'id': 'call_q5An6ZERzCELpDuQ40YUuu8U', 'type': 'function', 'function': {'name': 'send_email', 'arguments': '{"to":"mauromi@microsoft.com","subject":"Number of Flights","body":"Nr of flights between 2015-01-02 and 2021-04-04 is 2284"}'}}]}}

We need to run 1 tool call(s): [{'id': 'call_q5An6ZERzCELpDuQ40YUuu8U', 'type': 'function', 'function': {'name': 'send_email', 'arguments': '{"to":"mauromi@microsoft.com","subject":"Number of Flights","body":"Nr of flights between 2015-01-02 and 2021-04-04 is 2284"}'}}]

1 - Executing tool_call send_email (call_q5An6ZERzCELpDuQ40YUuu8U) >>> output: {"response": "<Response [200]>"}
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED


In [17]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
for m in messages.text_messages:
    print(m.text)

{'value': "I have written the number of flights between your cat's birth date (2015-01-02) and Easter 2021 (2021-04-04) into a file and sent an email to mauromi@microsoft.com with the information. The total number of flights is 2284.", 'annotations': []}
{'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.', 'annotations': []}


In [18]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: I have written the number of flights between your cat's birth date (2015-01-02) and Easter 2021 (2021-04-04) into a file and sent an email to mauromi@microsoft.com with the information. The total number of flights is 2284.


In [19]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
for rs in run_steps["data"]:
    print(rs, '\n')

{'id': 'step_3ICvLhYMKz7KyL7llXompat2', 'object': 'thread.run.step', 'created_at': 1735850111, 'run_id': 'run_8qHbfxr3tSfETSMHZwgWfJBM', 'assistant_id': 'asst_1TkHoUprIX7Xctg60VJ7sIPB', 'thread_id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735850112, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_2SxcWILvN2gLsBmEsLig4PTS'}}, 'usage': {'prompt_tokens': 612, 'completion_tokens': 64, 'total_tokens': 676}} 

{'id': 'step_aG2nWYbKwqgicFQFF5SGsuW6', 'object': 'thread.run.step', 'created_at': 1735850096, 'run_id': 'run_8qHbfxr3tSfETSMHZwgWfJBM', 'assistant_id': 'asst_1TkHoUprIX7Xctg60VJ7sIPB', 'thread_id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735850110, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type'

# START teardown

In [20]:
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_1TkHoUprIX7Xctg60VJ7sIPB', 'object': 'assistant', 'created_at': 1735850045, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string', 'description': 'No description'}}, 'required': ['date_1', 'date_2']}, 'strict': False}}, {'type': 'function',

In [21]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'object': 'thread', 'created_at': 1735850046, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_fiUbZyDgpPDX5TeI9XSPmrOz', 'object': 'thread.deleted', 'deleted': True}

In [22]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-assistant (asst_1TkHoUprIX7Xctg60VJ7sIPB)...


# HIC SUNT LEONES